In [21]:
from __future__ import print_function
import torch

### Chebyshev check for random matrix the accuracy by chebyshev

In [66]:
import numpy as np
import scipy
import scipy as sp
import scipy.sparse
from scipy.sparse.linalg import spsolve
from scipy.sparse import csr_matrix
from scipy.sparse import eye

### finding maximum and minimum singular value

In [67]:
A=torch.rand(10,10)
#A=torch.eye(100,100)
K=torch.matmul(A,A.t())
K_nump=K.numpy()
n=K_nump.shape[0]
#K_nump = np.random.randn(n, n)
#K_nump = K_nump.dot(K_nump.T)

In [68]:
K_nump.dot(x0).T.dot(x0)[0][0]

74.8549747525757

In [69]:
x0=np.random.rand(n,1)

In [70]:
u,s1,v=np.linalg.svd(K_nump)
eigs, vect=np.linalg.eig(K_nump)
lmax=max(eigs)
lmin=min(eigs)

In [71]:
print(s1[-1],s1[0])
print(lmin, lmax)

0.0088985795 26.814516
0.0088985795 26.814516


In [72]:
if (np.abs(lmin)<=1)and(np.abs(lmax)<=1):
    B=K_nump
    alpha=1
else:
    shift=0.1
    alpha=(lmax-lmin)/(lmax+lmin)+shift
    beta=(lmax+lmin)/2
    B=K_nump/(beta*alpha)-np.eye(n)/alpha

In [73]:
eigs_b, vect_b=np.linalg.eig(B)

In [84]:
eigs_b

array([ 0.90903614, -0.75588396, -0.78334812, -0.82582327, -0.84433711,
       -0.8603915 , -0.88881048, -0.90192128, -0.90903601, -0.90795984])

In [74]:
def T_cheb(m):
    if m==0:
        return 1
    else:
        c = np.zeros(m+1)
        c[m] = 1
        coef_ = np.polynomial.chebyshev.Chebyshev(c)
        return coef_
    
def roots(m):
    if m==0:
        return 1
    else:
        c = np.zeros(m+1)
        c[m] = 1
        roots = np.polynomial.chebyshev.chebroots(c)
        return roots
m = 100
root_cheb = roots(m+1)

In [75]:
print(alpha)
x=root_cheb
y=np.log(1+(alpha-shift)*x)

1.0993365406990052


In [76]:
s,log_det=np.linalg.slogdet(np.eye(n)+(alpha-shift)*B)

In [77]:
log_det

-17.131513333535487

In [78]:
coef=np.polynomial.chebyshev.chebfit(x, y,m)

In [79]:
log_det=0
batch_size=100
for i in range(batch_size):
    z=np.random.uniform(-1,1,(B.shape[0],1))
    z=z/np.linalg.norm(z)
    w=np.zeros((z.shape[0],m+1))
    w[:,0]=z.reshape(z.shape[0])
    w[:,1]=B.dot(z).reshape(z.shape[0])
    for j in range(2,m+1):
        w[:,j]=2*B.dot(w[:,j-1])-w[:,j-2]
    log_det+=z.T.dot(w).dot(coef)
        
log_det=log_det/batch_size 

In [80]:
n*log_det#+n*np.log(beta)

array([-16.87430692])